In [138]:
import pandas as pd
import requests
from io import BytesIO

In [139]:
# requests.get下載檔案
url = "https://www.ris.gov.tw/info-popudata/app/awFastDownload/file/y0s6-00000.xls/y0s6/00000/"
res = requests.get(url=url)


# 連結本身就會下載excel檔案，下載完後將其讀入
excel_data = BytesIO(res.content)
df = pd.read_excel(excel_data)


# 製作副本，並將「縣市」列表取出做成list
df_city_list = df.copy()
df_city_list = df_city_list.iloc[3:29]

columns = ["location", "population", "area(km^2)", "density"]
df_city_list.columns = columns
df_city_list["location"] = df_city_list["location"].str.replace(" ", "").str.replace("　", "").str.replace("※", "")

city_list = list(df_city_list["location"])


# 對總表修改欄位，去除掉多餘欄位（僅保留縣市及鄉鎮市區資料）
df.columns = columns
df.drop(index=range(0,42), axis=0, inplace=True)
df["location"] = df["location"].str.replace(" ", "").str.replace("　", "").str.replace("※", "")
df = df.reset_index(drop=True)


# 根據前面取得的縣市列表，取得「縣市」整區資料的位置並做成list
city_index = []
for city in city_list:
    idx = df.index[df["location"] == city].tolist()
    city_index.append(int(idx[0]))
    
    
# 新增一欄"city"並根據剛剛取得的縣市位置，將縣市填入，取得縣市+區兩個欄位
df["city"] = None

for n in range(0, len(city_index)):
    if n < len(city_index) - 1:
        df.loc[city_index[n]:city_index[n+1]-1, "city"] = city_list[n]
    else:
        df.loc[city_index[n]:, "city"] = city_list[n]
        
        
# 重新調整欄位名並重置index確保其索引正確
new_columns = ["district", "population", "area(km^2)", "density", "city"]
df.columns = new_columns
new_order = ["city", "district", "population", "area(km^2)", "density"]
df = df[new_order]


In [140]:
df

,city,district,population,area(km^2),density
0,新北市,新北市,4047001,2052.5667,1971.678192
1,新北市,板橋區,553538,23.1373,23924.053368
2,新北市,三重區,383355,16.317,23494.208494
3,新北市,中和區,405956,20.144,20152.700556
4,新北市,永和區,213742,5.7138,37408.029683
...,...,...,...,...,...
389,連江縣,北竿鄉,3062,9.9,309.292929
390,連江縣,莒光鄉,1518,4.7,322.978723
391,連江縣,東引鄉,1555,3.8,409.210526
392,東沙群島,東沙群島,…,2.38,…
